In [70]:
from bs4 import BeautifulSoup
from IPython.display import clear_output
from datetime import date, datetime, timedelta
from tqdm import tqdm
import urllib, urllib3
import requests, urllib.request, urllib.parse, csv, re, time, os, numpy as np, pandas as pd, json
from datetime import date, datetime, timedelta

key_minvv23 = '290dac7fcac7c640c069ced9e576bc00'

In [2]:
company_basicurl = 'http://kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json?key={key}&companyNm={company}'
company_detailurl = 'http://kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyInfo.json?key={key}&companyCd={companycode}'
warner_search = json.loads(urllib.request.urlopen(company_basicurl.format(key=key_minvv23, company='Warner')).read())
warner_codes = [each['companyCd'] for each in warner_search['companyListResult']['companyList'][:2]]

In [3]:
movie_codes = []

for each_companycode in warner_codes :
    warner_info = json.loads(urllib.request.urlopen(company_detailurl.format(key=key_minvv23, companycode=each_companycode)).read())
    for each in warner_info['companyInfoResult']['companyInfo']['filmos'] :
        movie_codes.append(int(each['movieCd']))

movie_codes = list(filter(lambda x : x>20100000, set(movie_codes)))

In [4]:
movie_info = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movie_code}'

In [5]:
wb_weblist = []
for each_code in movie_codes :
    movieapi = json.loads(urllib.request.urlopen(movie_info.format(key=key_minvv23, movie_code=each_code)).read())
    wb_weblist.append([movieapi['movieInfoResult']['movieInfo']['movieNmEn'], movieapi['movieInfoResult']['movieInfo']['movieCd']])

In [6]:
wb_csvlist = pd.read_csv('WB_Movies.csv')
wb_csvlist['theater_release'] = np.vectorize(lambda x : ''.join(re.findall(r'\d+', x)))(wb_csvlist['theater_release'].values)
wb_csvlist['vod_release'] = np.vectorize(lambda x : ''.join(re.findall(r'\d+', x)))(wb_csvlist['vod_release'].values)

In [7]:
for each_movie in wb_weblist :
    each_movie.append(''.join(each_movie[0].lower().split()))

In [44]:
listforcodejoin = []
for each_csvmovie in wb_csvlist.name :
    shortcsv = ''.join(each_csvmovie.lower().split())
    for fullweb, codeweb, shortweb in wb_weblist :
        if shortweb == shortcsv :
            listforcodejoin.append((each_csvmovie, codeweb))
listforcodejoin = pd.DataFrame.from_records(listforcodejoin)
listforcodejoin.columns = ['name', 'movieCd']

In [50]:
#pd.merge(wb_csvlist, listforcodejoin, how='outer').to_csv('UPDATED_WB_Movies.csv')

In [80]:
upd_wbcsvlist = pd.read_csv('UPDATED_WB_Movies.csv')
upd_wbcsvlist.head()

,Unnamed: 0,name,admission,theater_release,vod_release,movieCd
0,0,American Sniper,345782,20150114,20150416,20143754
1,1,Jupiter Ascending,309842,20150205,20150507,20149037
2,2,Run all night,289534,20150319,20150618,20157041
3,3,Mad Max: Fury Road,3884129,20150514,20150813,20144741
4,4,San Andreas,1716455,20150603,20150903,20144682


In [86]:
boxofficeurl = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={key}&targetDt={date}'
date_list = [(datetime.strptime(str(20150101),'%Y%m%d') + timedelta(days=x)).strftime('%Y%m%d')
             for x in range(0, (datetime.strptime(str(20190531),'%Y%m%d')-datetime.strptime(str(20150101),'%Y%m%d')).days+1)]

In [154]:
dailyboxoffice_list = []

for each_date in tqdm(date_list[25:50]) :
    dailyrecord = json.loads(urllib.request.urlopen(boxofficeurl.format(key=key_minvv23, date=each_date)).read())
    for each_boxoffice in dailyrecord['boxOfficeResult']['dailyBoxOfficeList'] :
        moviedict = {}
        if int(each_boxoffice['movieCd']) in list(upd_wbcsvlist['movieCd']) :
            moviedict['date'] = each_date
            moviedict['movieCd'] = each_boxoffice['movieCd']
            moviedict['salesAmt'] = each_boxoffice['salesAmt']
            dailyboxoffice_list.append(moviedict)

100%|██████████| 25/25 [00:16<00:00,  1.54it/s]


In [155]:
dailyboxoffice_list

[{'date': '20150126', 'movieCd': '20143754', 'salesAmt': '48168200'},
 {'date': '20150127', 'movieCd': '20143754', 'salesAmt': '49987700'},
 {'date': '20150202', 'movieCd': '20143754', 'salesAmt': '34449900'},
 {'date': '20150204', 'movieCd': '20143754', 'salesAmt': '39147700'},
 {'date': '20150205', 'movieCd': '20149037', 'salesAmt': '376906800'},
 {'date': '20150206', 'movieCd': '20149037', 'salesAmt': '394072943'},
 {'date': '20150206', 'movieCd': '20143754', 'salesAmt': '27869000'},
 {'date': '20150207', 'movieCd': '20149037', 'salesAmt': '788448636'},
 {'date': '20150208', 'movieCd': '20149037', 'salesAmt': '660648900'},
 {'date': '20150209', 'movieCd': '20149037', 'salesAmt': '152371300'},
 {'date': '20150209', 'movieCd': '20143754', 'salesAmt': '15900600'},
 {'date': '20150210', 'movieCd': '20149037', 'salesAmt': '147095100'},
 {'date': '20150211', 'movieCd': '20149037', 'salesAmt': '27052500'}]